# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [6]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-dfdde0393e-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Economy of EU States") 

Consider the following exploratory information need:

> Compare businesses across different sectors and types in E.U. countries

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wd:Q458`      | E.U.         | node |
| `wd:Q142`      | France       | node      |
| `wd:Q4830453`  | Business     | node      |
| `wd:Q6881511`  | Enterprise   | node      |
| `wd:Q29110228` | AXA          | node |
| `wd:Q43183`    | insurance    | node |




Also consider

```
{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
}



```

is the BGP to retrieve all **french enterprises and businesses**

## Workload Goals

1. Identify the BGP for obtaining the type, legal form, and industry of a company

2. Identify the BGP to retrieve all companies owned by a company located in a EU country

3. Which company has the largest presence in E.U.?

4. Companies have different 'legal forms', compare the number of companies divided in different legal forms

5. Analyze the number of companies per type, legal form, and industry in each state
 
   5.1 What are the top-3 legal form in E.U.? 
   
   5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each country for a given legal form?
   
   5.3 Which business in each country owns more businesses in other E.U. countries?
   
   5.4 What can we say about industry sectors in various countries?


### Point 1 Start

In [7]:
# start your workflow here

In [8]:
queryString = """
SELECT COUNT(*)
WHERE { 

{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
}
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '4577')]


1

In [9]:
#Print all outgoing properties of a company
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 
    
    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company ?property ?object.
    ?property <http://schema.org/name> ?propertyName.
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P8868'), ('propertyName', 'AAA hotel ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8783'), ('propertyName', 'AAGM person or institution ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2179'), ('propertyName', 'ACM Classification Code (2012)')]
[('property', 'http://www.wikidata.org/prop/direct/P2472'), ('propertyName', 'ACMA Radiocommunications Client Number')]
[('property', 'http://www.wikidata.org/prop/direct/P6981'), ('propertyName', 'ACNP journal ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3901'), ('propertyName', 'ADAGP artist ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2342'), ('propertyName', 'AGORHA person/institution ID')]
[('property', 'http://www.wikidata.org/prop/direct/P6269'), ('propertyName', 'API endpoint')]
[('property', 'http://www.wikidata.org/prop/direct/P9226'), ('propertyName', 'ARAE ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3642'), ('proper

1702

In [10]:
#Print all ingoing properties of a company
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 
    
    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?object ?property ?company.
    ?property <http://schema.org/name> ?propertyName.
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P880'), ('propertyName', 'CPU')]
[('property', 'http://www.wikidata.org/prop/direct/P1629'), ('propertyName', 'Wikidata item of this property')]
[('property', 'http://www.wikidata.org/prop/direct/P1855'), ('propertyName', 'Wikidata property example')]
[('property', 'http://www.wikidata.org/prop/direct/P1204'), ('propertyName', "Wikimedia portal's main topic")]
[('property', 'http://www.wikidata.org/prop/direct/P5514'), ('propertyName', 'accredited by')]
[('property', 'http://www.wikidata.org/prop/direct/P1817'), ('propertyName', 'addressee')]
[('property', 'http://www.wikidata.org/prop/direct/P3032'), ('propertyName', 'adjacent building')]
[('property', 'http://www.wikidata.org/prop/direct/P197'), ('propertyName', 'adjacent station')]
[('property', 'http://www.wikidata.org/prop/direct/P6291'), ('propertyName', 'advertises')]
[('property', 'http://www.wikidata.org/prop/direct/P1416'), ('propertyName', 'affiliation')]
[('property

341

In [9]:
#Get the property that connects "AXA" with "insurance"
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    wd:Q29110228 ?property wd:Q43183.
    ?property <http://schema.org/name> ?propertyName.
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [6]:
#Get the property that connects "insurance" with "AXA" 
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    wd:Q43183 ?property wd:Q29110228.
    ?property <http://schema.org/name> ?propertyName.
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [10]:
#Get the indistry and legal form of "AXA"
queryString = """
SELECT DISTINCT ?indistryName ?legalFormName
WHERE { 

    wd:Q29110228 wdt:P452 ?indistry.
    ?indistry <http://schema.org/name> ?indistryName.
    
    wd:Q29110228 wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.
    
} 
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [11]:
#Get the class of "AXA"
queryString = """
SELECT DISTINCT ?class ?className
WHERE { 

    wd:Q29110228 wdt:P31 ?class.
    ?class <http://schema.org/name> ?className.
    
} 
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q4830453'), ('className', 'business')]
[('class', 'http://www.wikidata.org/entity/Q6881511'), ('className', 'enterprise')]


2

In [12]:
#Get the class of "insurance"
queryString = """
SELECT DISTINCT ?class ?className
WHERE { 

    wd:Q43183 wdt:P31 ?class.
    ?class <http://schema.org/name> ?className.
    
} 
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q8187769'), ('className', 'economic activity')]
[('class', 'http://www.wikidata.org/entity/Q268592'), ('className', 'industry')]


2

In [13]:
#Get all "AXA" outgoing properties
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    wd:Q29110228 ?property ?object.
    ?property <http://schema.org/name> ?propertyName.
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2622'), ('propertyName', 'Companies House company ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5785'), ('propertyName', 'EU Research participant ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2657'), ('propertyName', 'EU Transparency Register ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3608'), ('propertyName', 'EU VAT number')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('propertyName', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P1278'), ('propertyName', 'Legal Entity Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P4776'), ('propertyName', 'MAC Address Block Large ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1320'), ('propertyName', 'OpenCorporates ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3500'), ('propertyName', 'Ringgold ID')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('propertyName', 'countr

17

In [18]:
#Get "AXA" name
queryString = """
SELECT DISTINCT ?name
WHERE { 

    wd:Q29110228 <http://schema.org/name> ?name.
    
} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'Panasonic Europe Ltd')]


1

In [14]:
#Get "AXA" legal form
queryString = """
SELECT DISTINCT ?object ?objectName
WHERE { 

    wd:Q29110228 wdt:P1454 ?object.
    ?object <http://schema.org/name> ?objectName.
    
} 
ORDER BY ?objectName
"""

print("Results")
run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q6832945'), ('objectName', 'private company limited by shares')]


1

In [15]:
# Identify the BGP for obtaining the type, legal form, and industry of a company
queryString = """
SELECT DISTINCT ?companyName ?propertyName ?objectName
WHERE { 

    {
        ?company wdt:P31 wd:Q6881511. 
    } 
    UNION
    {
        ?company wdt:P31 wd:Q4830453. 
    }
    
    ?company ?property ?object.
    FILTER(?property = wdt:P1454 || ?property = wdt:P452)
    
    ?company <http://schema.org/name> ?companyName.
    ?property <http://schema.org/name> ?propertyName.
    ?object <http://schema.org/name> ?objectName.
    
} 
ORDER BY ?companyName
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('companyName', '"APR"'), ('propertyName', 'legal form'), ('objectName', 'společnost s ručením omezeným')]
[('companyName', '"ATOEX"'), ('propertyName', 'legal form'), ('objectName', 'Spoločnosť s ručením obmedzeným')]
[('companyName', '"Agentura REPRE" společnost s ručením omezeným'), ('propertyName', 'legal form'), ('objectName', 'společnost s ručením omezeným')]
[('companyName', '"CZ TOP TRADE s.r.o."'), ('propertyName', 'legal form'), ('objectName', 'společnost s ručením omezeným')]
[('companyName', '"ELVE svěřenský fond"'), ('propertyName', 'legal form'), ('objectName', 'svěřenský fond')]
[('companyName', '"Hlásek, spol. s.r.o."'), ('propertyName', 'legal form'), ('objectName', 'společnost s ručením omezeným')]
[('companyName', '"KF DOMENA" SPÓŁKA Z OGRANICZONA ODPOWIEDZIALNOSCIA'), ('propertyName', 'legal form'), ('objectName', 'Sp. z o.o.')]
[('companyName', '"Kosmos" Tabak- & Cigarettenfabrik H. F. Wolf'), ('propertyName', 'legal form'), ('objectName', 'privately held 

100

### Point 1 End
DESCRIPTION: I started by checking all the ingoing and outgoing properties of a company. A company is an entity that is instance of “business” wd:Q4830453 or “enterprise” wd:Q6881511. 
Then I perform some exploration about AXA company and Esurance industry.
At the end I kept the following relations “legal form” wdt:P1454 and “industry” wdt:P452.

RESULT: The result shows the list of the companies in the database ordered by name along with the legal form and industry if present.

### Point 2 Start

In [16]:
# Return the class of "France"
queryString = """
SELECT DISTINCT ?class ?className
WHERE { 

    wd:Q142 wdt:P31 ?class.
    ?class <http://schema.org/name> ?className.
    
} 

"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q3624078'), ('className', 'sovereign state')]
[('class', 'http://www.wikidata.org/entity/Q51576574'), ('className', 'Mediterranean country')]
[('class', 'http://www.wikidata.org/entity/Q6256'), ('className', 'country')]
[('class', 'http://www.wikidata.org/entity/Q7270'), ('className', 'republic')]


4

In [17]:
# Return the relations between an instance of "country" and "E.U."
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    ?country wdt:P31 wd:Q6256.
    ?country ?property wd:Q458.
    ?property <http://schema.org/name> ?propertyName.
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P530'), ('propertyName', 'diplomatic relation')]
[('property', 'http://www.wikidata.org/prop/direct/P463'), ('propertyName', 'member of')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('propertyName', 'part of')]
[('property', 'http://www.wikidata.org/prop/direct/P47'), ('propertyName', 'shares border with')]


4

In [18]:
# Return the relations between "France" and "E.U."
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    wd:Q142 ?property wd:Q458.
    ?property <http://schema.org/name> ?propertyName.
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P463'), ('propertyName', 'member of')]


1

In [31]:
# Return all countries meber of "E.U."
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE { 

    ?country wdt:P463 wd:Q458.
    ?country <http://schema.org/name> ?countryName.
    
    
} 
ORDER BY ?countryName
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('countryName', 'Bulgaria')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('countryName', 'Estonia')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('countryName', 'Finland')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('countryName', 'Greece')]
[('country', 'http:/

28

In [11]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MIN(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?legalFormName ?marketCap ?companyName 
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('legalFormName', 'akciová společnost'), ('marketCap', '7100000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('legalFormName', 'aktieselskab'), ('marketCap', '25256000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'France'), ('companyName', 'Prodways Group'), ('legalFormName', 'S.A.'), ('marketCap', '157659497')]
[('euCountryName', 'France'), ('companyName', 'Renault'), ('legalFormName', 'société anonyme'), ('ma

18

### Point 2 End
DESCRIPTION: I made some test to get the E.U. countries. Then I used the property “owned by” wdt:P127.
In the final query I searched for all companies located in a E.U. country and then I returned the companies owned by these companies.


RESULT: The result shows a list of 100 companies owned by a company located in a E.U. country

### Point 3 Start

In [12]:
#Get all subsidiary of "AXA"
queryString = """
SELECT DISTINCT ?subsidiaryName
WHERE { 

    wd:Q29110228 wdt:P355 ?subsidiary.
    ?subsidiary <http://schema.org/name> ?subsidiaryName.
    
} 
ORDER BY ?subsidiaryName
"""

print("Results")
run_query(queryString)

Results
[('subsidiaryName', 'Panasonic AVC Networks Czech')]


1

In [13]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MIN(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?legalFormName ?marketCap ?companyName 
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('legalFormName', 'akciová společnost'), ('marketCap', '7100000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('legalFormName', 'aktieselskab'), ('marketCap', '25256000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'France'), ('companyName', 'Prodways Group'), ('legalFormName', 'S.A.'), ('marketCap', '157659497')]
[('euCountryName', 'France'), ('companyName', 'Renault'), ('legalFormName', 'société anonyme'), ('ma

18

### Point 3 End
DESCRIPTION: For this query I considered all E.U. companies owned by a company. Then to get which company is more present in Europe I counted how many companies in E.U. this company owns.


RESULT: The result shows that BlackRock is the one that owns more companies in E.U. We can see that RAI is the first Italian most present company in Europe.

### Point 4 Start

In [49]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MIN(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?legalFormName ?marketCap ?companyName 
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('legalFormName', 'akciová společnost'), ('marketCap', '7100000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('legalFormName', 'aktieselskab'), ('marketCap', '25256000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'France'), ('companyName', 'Prodways Group'), ('legalFormName', 'S.A.'), ('marketCap', '157659497')]
[('euCountryName', 'France'), ('companyName', 'Renault'), ('legalFormName', 'société anonyme'), ('ma

18

### Point 4 End
RESULT: The result shows the 100 legal forms most adopted by companies. The most used is “společnost s ručením omezeným” wd:Q15646299 with 13075 companies. The most used Italian legal form is “società per azioni” wd:Q3742388 with 349 companies.

### Point 5.1 Start

In [49]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MIN(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?legalFormName ?marketCap ?companyName 
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('legalFormName', 'akciová společnost'), ('marketCap', '7100000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('legalFormName', 'aktieselskab'), ('marketCap', '25256000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'France'), ('companyName', 'Prodways Group'), ('legalFormName', 'S.A.'), ('marketCap', '157659497')]
[('euCountryName', 'France'), ('companyName', 'Renault'), ('legalFormName', 'société anonyme'), ('ma

18

### Point 5.1 End
RESULT: This query is the same as the previous one but the considered companies are locate in a E.U. country.

### Point 5.2 Start

#### Point 5.2.1 Start

In [6]:
# Get info about "operating income"
queryString = """
SELECT DISTINCT ?operatingIncome ?operatingIncomeName
WHERE { 

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P3362 ?operatingIncome.
    ?operatingIncome <http://schema.org/name> ?operatingIncomeName.
    

    
}
ORDER BY ?operatingIncomeName
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [7]:
# Get info about "operating income"
queryString = """
SELECT DISTINCT ?operatingIncome 
WHERE { 

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P3362 ?operatingIncome.
    
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('operatingIncome', '422699000')]
[('operatingIncome', '-2300000000')]
[('operatingIncome', '98433000')]
[('operatingIncome', '14000000000')]
[('operatingIncome', '-855000000')]
[('operatingIncome', '1190000000')]
[('operatingIncome', '30613000000')]
[('operatingIncome', '898187000')]
[('operatingIncome', '56000000')]
[('operatingIncome', '25898000')]
[('operatingIncome', '43500000')]
[('operatingIncome', '1949000000')]
[('operatingIncome', '-1661000')]
[('operatingIncome', '2710000000')]
[('operatingIncome', '164000000')]
[('operatingIncome', '3266000')]
[('operatingIncome', '24071000000')]
[('operatingIncome', '5664000000')]
[('operatingIncome', '-571000')]
[('operatingIncome', '810225000')]
[('operatingIncome', '132423000')]
[('operatingIncome', '850291000')]
[('operatingIncome', '19759000000')]
[('operatingIncome', '2206212000')]
[('operatingIncome', '5441000000')]
[('operatingIncome', '6800000000')]
[('operatingIncome', '15996000000')]
[('operatingIncome', '50998000000')]

100

In [9]:
# Get info about "source of income"
queryString = """
SELECT DISTINCT ?sourceOfIncome ?sourceOfIncomeName
WHERE { 

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P2770 ?sourceOfIncome.
    ?sourceOfIncome <http://schema.org/name> ?sourceOfIncomeName.
    
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sourceOfIncome', 'http://www.wikidata.org/entity/Q1148747'), ('sourceOfIncomeName', 'payment')]
[('sourceOfIncome', 'http://www.wikidata.org/entity/Q1254596'), ('sourceOfIncomeName', 'software as a service')]
[('sourceOfIncome', 'http://www.wikidata.org/entity/Q2424752'), ('sourceOfIncomeName', 'product')]
[('sourceOfIncome', 'http://www.wikidata.org/entity/Q322948'), ('sourceOfIncomeName', 'subscription')]
[('sourceOfIncome', 'http://www.wikidata.org/entity/Q37038'), ('sourceOfIncomeName', 'advertising')]
[('sourceOfIncome', 'http://www.wikidata.org/entity/Q7406919'), ('sourceOfIncomeName', 'service')]
[('sourceOfIncome', 'http://www.wikidata.org/entity/Q624902'), ('sourceOfIncomeName', 'online advertising')]
[('sourceOfIncome', 'http://www.wikidata.org/entity/Q4054833'), ('sourceOfIncomeName', 'subscription business model')]
[('sourceOfIncome', 'http://www.wikidata.org/entity/Q183480'), ('sourceOfIncomeName', 'Google AdSense')]
[('sourceOfIncome', 'http://www.wikidata.org/

10

In [10]:
# Get info about "common equity tier 1 capital rateo"
queryString = """
SELECT DISTINCT ?CETR
WHERE { 

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P2663 ?CETR.

    
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('CETR', '10')]
[('CETR', '11.6')]
[('CETR', '14.9')]
[('CETR', '20.2')]
[('CETR', '11.66')]
[('CETR', '19.1')]
[('CETR', '19.5')]
[('CETR', '13.3')]
[('CETR', '16.5')]
[('CETR', '13.7')]
[('CETR', '20.49')]
[('CETR', '11.8')]
[('CETR', '13.03')]
[('CETR', '18')]
[('CETR', '19')]
[('CETR', '12.07')]
[('CETR', '20.13')]
[('CETR', '20.53')]
[('CETR', '22.31')]
[('CETR', '15.31')]
[('CETR', '14.78')]
[('CETR', '15.1')]
[('CETR', '27.8')]
[('CETR', '23')]
[('CETR', '18.79')]
[('CETR', '14.65')]
[('CETR', '37.25')]
[('CETR', '11.45')]


28

In [11]:
# Get info about "market capitalization"
queryString = """
SELECT DISTINCT ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P2226 ?marketCap.

    
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('marketCap', '7000000000')]
[('marketCap', '1239900000')]
[('marketCap', '17870000000')]
[('marketCap', '60000000000')]
[('marketCap', '265300000000')]
[('marketCap', '286200000000')]
[('marketCap', '6858000000')]
[('marketCap', '172750000000')]
[('marketCap', '38105000000')]
[('marketCap', '344490000000')]
[('marketCap', '232000000000')]
[('marketCap', '31000000000')]
[('marketCap', '200000000000')]
[('marketCap', '223140000000')]
[('marketCap', '139800000000')]
[('marketCap', '111760000')]
[('marketCap', '237363000000')]
[('marketCap', '32580000000')]
[('marketCap', '11643000000')]
[('marketCap', '6882000000')]
[('marketCap', '486000000000')]
[('marketCap', '62500000000')]
[('marketCap', '156700000000')]
[('marketCap', '176347231087.57')]
[('marketCap', '8270000000')]
[('marketCap', '9431200000')]
[('marketCap', '26560000000')]
[('marketCap', '2458000000000')]
[('marketCap', '26257000000')]
[('marketCap', '3100000000000')]
[('marketCap', '185400000000')]
[('marketCap', '102

100

In [13]:
# Get info about "assets under management"
queryString = """
SELECT DISTINCT ?assetsUM
WHERE { 

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P4103 ?assetsUM.

    
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('assetsUM', '1521000000000')]
[('assetsUM', '1220000000000')]
[('assetsUM', '3518000000000')]
[('assetsUM', '6100000000000')]
[('assetsUM', '335000000000')]
[('assetsUM', '657000000000')]
[('assetsUM', '1145000000000')]
[('assetsUM', '973000000000')]
[('assetsUM', '229992000000')]
[('assetsUM', '8816622000')]
[('assetsUM', '7318000000000')]
[('assetsUM', '1700000000000')]
[('assetsUM', '239825000000')]
[('assetsUM', '783400000000')]
[('assetsUM', '3054000000000')]
[('assetsUM', '2511000000000')]
[('assetsUM', '2400000000000')]
[('assetsUM', '3319000000000')]
[('assetsUM', '2729000000000')]
[('assetsUM', '628500000000')]
[('assetsUM', '647900000000')]
[('assetsUM', '1791000000000')]
[('assetsUM', '795100000000')]
[('assetsUM', '1143000000000')]
[('assetsUM', '564300000000')]
[('assetsUM', '802000000000')]
[('assetsUM', '1010000000000')]
[('assetsUM', '600000000000')]
[('assetsUM', '565700000000')]
[('assetsUM', '1090000000000')]
[('assetsUM', '1221000000000')]
[('assetsUM', '1

65

In [14]:
# Get info about "total assets"
queryString = """
SELECT DISTINCT ?totalAssets
WHERE { 

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P2403 ?totalAssets.

    
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('totalAssets', '7824063000')]
[('totalAssets', '2793294000000')]
[('totalAssets', '603000000000')]
[('totalAssets', '2165963000')]
[('totalAssets', '342889000000')]
[('totalAssets', '13329000000')]
[('totalAssets', '694626000')]
[('totalAssets', '10518247000')]
[('totalAssets', '2203115000')]
[('totalAssets', '927000000')]
[('totalAssets', '3420638000')]
[('totalAssets', '101485000')]
[('totalAssets', '2610019000')]
[('totalAssets', '228300000000')]
[('totalAssets', '121697000')]
[('totalAssets', '271558000000')]
[('totalAssets', '2933216465722')]
[('totalAssets', '91102000000')]
[('totalAssets', '18048071000')]
[('totalAssets', '10945322000')]
[('totalAssets', '12000000000')]
[('totalAssets', '707443000000')]
[('totalAssets', '114995477000')]
[('totalAssets', '1280000000')]
[('totalAssets', '111993000000')]
[('totalAssets', '332973000')]
[('totalAssets', '845000000000')]
[('totalAssets', '3310735000')]
[('totalAssets', '90707000000')]
[('totalAssets', '10484000000')]
[('tota

100

In [19]:
# For which companies is defined some form of income or market capitalization or total assets?
# Used properties "operating income", "source of income", "market capitalization", "assets under management", "total assets"
queryString = """
SELECT DISTINCT ?company ?companyName
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company ?property ?object.
    FILTER(?property = wdt:P3362 || ?property = wdt:P2770 
    || ?property = wdt:P2226 || ?property = wdt:P4103 || ?property = wdt:P2403)

    
}
ORDER BY ?companyName
"""

print("Results")
run_query(queryString)

Results
[('company', 'http://www.wikidata.org/entity/Q162472'), ('companyName', '1. FK Příbram')]
[('company', 'http://www.wikidata.org/entity/Q59754453'), ('companyName', '1.Tatranská, akciová spoločnosť')]
[('company', 'http://www.wikidata.org/entity/Q222045'), ('companyName', '2K Czech')]
[('company', 'http://www.wikidata.org/entity/Q226889'), ('companyName', '36,6')]
[('company', 'http://www.wikidata.org/entity/Q4636301'), ('companyName', '3D Systems')]
[('company', 'http://www.wikidata.org/entity/Q94454239'), ('companyName', '4Fund')]
[('company', 'http://www.wikidata.org/entity/Q102076789'), ('companyName', 'A.B.R. Holding')]
[('company', 'http://www.wikidata.org/entity/Q1888809'), ('companyName', 'A1 Belarus')]
[('company', 'http://www.wikidata.org/entity/Q11683986'), ('companyName', 'AB')]
[('company', 'http://www.wikidata.org/entity/Q20056888'), ('companyName', 'AB - Credit')]
[('company', 'http://www.wikidata.org/entity/Q30258448'), ('companyName', 'ABB')]
[('company', 'http:

1587

#### Point 5.2.1 End

#### Point 5.2.2 Start

In [49]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MIN(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?legalFormName ?marketCap ?companyName 
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('legalFormName', 'akciová společnost'), ('marketCap', '7100000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('legalFormName', 'aktieselskab'), ('marketCap', '25256000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'France'), ('companyName', 'Prodways Group'), ('legalFormName', 'S.A.'), ('marketCap', '157659497')]
[('euCountryName', 'France'), ('companyName', 'Renault'), ('legalFormName', 'société anonyme'), ('ma

18

In [59]:
# What is the min in each country?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P31 wd:Q6256.
    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.


    {
        SELECT ?euCountryIn (MIN(?marketCapIn) AS ?minMarketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P31 wd:Q6256.
            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?minMarketCapitalizationIn)
    
}
ORDER BY ?euCountryName ?companyName
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('marketCap', '7100000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('marketCap', '25256000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('marketCap', '26257000000')]
[('euCountryName', 'France'), ('companyName', 'Prodways Group'), ('marketCap', '157659497')]
[('euCountryName', 'Greece'), ('companyName', 'Macedonian Thrace Brewery'), ('marketCap', '3896025.7')]
[('euCountryName', 'Italy'), ('companyName', 'Pirelli'), ('marketCap', '7130000000')]
[('euCountryName', 'Kingdom of the Netherlands'), ('companyName', 'Randstad N.V.'), ('marketCap', '9431200000')]
[('euCountryName', 'Sweden'), ('companyName', 'Trelleborg AB'), ('marketCap', '37774000000')]
[('euCountryName', 'United Kingdom'), ('companyName', 'Lloyds Banking Group'), ('marketCap', '47000000000')]


10

In [60]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P31 wd:Q6256.
    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MIN(?marketCapIn) AS ?minMarketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P31 wd:Q6256.
            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?minMarketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?companyName
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('legalFormName', 'akciová společnost'), ('marketCap', '7100000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('legalFormName', 'aktieselskab'), ('marketCap', '25256000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'France'), ('companyName', 'Prodways Group'), ('legalFormName', 'S.A.'), ('marketCap', '157659497')]
[('euCountryName', 'France'), ('companyName', 'Renault'), ('legalFormName', 'société anonyme'), ('ma

13

In [63]:
# What is the min, max, and average in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P31 wd:Q6256.
    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (AVG(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P31 wd:Q6256.
            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    UNION
    {
        SELECT ?euCountryIn ?legalFormIn (MIN(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P31 wd:Q6256.
            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    UNION
    {
        SELECT ?euCountryIn ?legalFormIn (MAX(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P31 wd:Q6256.
            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?companyName ?legalFormName ?marketCap
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('legalFormName', 'akciová společnost'), ('marketCap', '7100000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'ČEZ Group'), ('legalFormName', 'akciová společnost'), ('marketCap', '286200000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('legalFormName', 'aktieselskab'), ('marketCap', '25256000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Ørsted'), ('legalFormName', 'aktieselskab'), ('marketCap', '289600000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Banka

19

In [64]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MIN(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?companyName ?legalFormName
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('legalFormName', 'akciová společnost'), ('marketCap', '7100000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('legalFormName', 'aktieselskab'), ('marketCap', '25256000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'France'), ('companyName', 'Prodways Group'), ('legalFormName', 'S.A.'), ('marketCap', '157659497')]
[('euCountryName', 'France'), ('companyName', 'Renault'), ('legalFormName', 'société anonyme'), ('ma

13

In [49]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MIN(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?legalFormName ?marketCap ?companyName 
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'PFNonwovens'), ('legalFormName', 'akciová společnost'), ('marketCap', '7100000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Maersk'), ('legalFormName', 'aktieselskab'), ('marketCap', '25256000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'France'), ('companyName', 'Prodways Group'), ('legalFormName', 'S.A.'), ('marketCap', '157659497')]
[('euCountryName', 'France'), ('companyName', 'Renault'), ('legalFormName', 'société anonyme'), ('ma

18

#### Point 5.2.2 End

#### Point 5.2.3 Start

In [65]:
# What is the max in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P31 wd:Q6256.
    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MAX(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P31 wd:Q6256.
            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?companyName ?legalFormName
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'ČEZ Group'), ('legalFormName', 'akciová společnost'), ('marketCap', '286200000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Ørsted'), ('legalFormName', 'aktieselskab'), ('marketCap', '289600000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'France'), ('companyName', 'Carrefour'), ('legalFormName', 'S.A.'), ('marketCap', '10203000000')]
[('euCountryName', 'France'), ('companyName', 'TotalEnergies'), ('legalFormName', 'société anonyme'), 

13

In [50]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (MAX(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?legalFormName ?marketCap ?companyName 
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Czech Republic'), ('companyName', 'ČEZ Group'), ('legalFormName', 'akciová společnost'), ('marketCap', '286200000000')]
[('euCountryName', 'Denmark'), ('companyName', 'Ørsted'), ('legalFormName', 'aktieselskab'), ('marketCap', '289600000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'France'), ('companyName', 'Carrefour'), ('legalFormName', 'S.A.'), ('marketCap', '10203000000')]
[('euCountryName', 'France'), ('companyName', 'TotalEnergies'), ('legalFormName', 'société anonyme'), 

18

#### Point 5.2.3 End

#### Point 5.2.4 Start

In [66]:
# What is the average in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT DISTINCT ?euCountryName ?companyName ?legalFormName ?marketCap
WHERE { 

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType= wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P31 wd:Q6256.
    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P2226 ?marketCap.
    
    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.


    {
        SELECT ?euCountryIn ?legalFormIn (AVG(?marketCapIn) AS ?marketCapitalizationIn)
        WHERE {

            ?companyIn wdt:P31 ?companyTypeIn.
            FILTER(?companyTypeIn = wd:Q6881511 || ?companyTypeIn = wd:Q4830453)

            ?euCountryIn wdt:P31 wd:Q6256.
            ?euCountryIn wdt:P463 wd:Q458.

            ?companyIn wdt:P17 ?euCountryIn.
            
            ?companyIn wdt:P1454 ?legalFormIn.

            ?companyIn wdt:P2226 ?marketCapIn.

        }
        GROUP BY ?euCountryIn ?legalFormIn
    }
    
    FILTER(?euCountry = ?euCountryIn && ?marketCap = ?marketCapitalizationIn && ?legalForm = ?legalFormIn)
    
}
ORDER BY ?euCountryName ?companyName ?legalFormName
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('companyName', 'OMV'), ('legalFormName', 'Aktiengesellschaft'), ('marketCap', '10960000000')]
[('euCountryName', 'Austria'), ('companyName', 'Strabag'), ('legalFormName', 'Societas Europaea'), ('marketCap', '3500000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nokia'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCap', '26257000000')]
[('euCountryName', 'Finland'), ('companyName', 'Nordea'), ('legalFormName', 'Bankaktiebolag'), ('marketCap', '415000000000')]
[('euCountryName', 'Greece'), ('companyName', 'Macedonian Thrace Brewery'), ('legalFormName', 'S.A.'), ('marketCap', '3896025.7')]
[('euCountryName', 'Italy'), ('companyName', 'Pirelli'), ('legalFormName', 'società per azioni'), ('marketCap', '7130000000')]
[('euCountryName', 'Kingdom of the Netherlands'), ('companyName', 'Randstad N.V.'), ('legalFormName', 'Naamloze vennootschap'), ('marketCap', '9431200000')]


7

#### Point 5.2.4 End

In [57]:
# What is the min in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT ?euCountryName ?legalFormName (MIN(?marketCap) AS ?marketCapitalization)
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.

    ?company wdt:P2226 ?marketCap.

}
GROUP BY ?euCountryName ?legalFormName ?companyName
ORDER BY ?euCountryName ?legalFormName
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('legalFormName', 'Aktiengesellschaft'), ('marketCapitalization', '10960000000')]
[('euCountryName', 'Austria'), ('legalFormName', 'Societas Europaea'), ('marketCapitalization', '3500000000')]
[('euCountryName', 'Czech Republic'), ('legalFormName', 'akciová společnost'), ('marketCapitalization', '7100000000')]
[('euCountryName', 'Denmark'), ('legalFormName', 'aktieselskab'), ('marketCapitalization', '25256000000')]
[('euCountryName', 'Finland'), ('legalFormName', 'Bankaktiebolag'), ('marketCapitalization', '415000000000')]
[('euCountryName', 'Finland'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCapitalization', '26257000000')]
[('euCountryName', 'France'), ('legalFormName', 'S.A.'), ('marketCapitalization', '157659497')]
[('euCountryName', 'France'), ('legalFormName', 'société anonyme'), ('marketCapitalization', '12474000000')]
[('euCountryName', 'Germany'), ('legalFormName', 'Aktiengesellschaft'), ('marketCapitalization', '111760000')]
[(

17

In [60]:
# What is the max in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT ?euCountryName ?legalFormName (MAX(?marketCap) AS ?marketCapitalization)
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.

    ?company wdt:P2226 ?marketCap.

}
GROUP BY ?euCountryName ?legalFormName
ORDER BY ?euCountryName ?legalFormName
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('legalFormName', 'Aktiengesellschaft'), ('marketCapitalization', '10960000000')]
[('euCountryName', 'Austria'), ('legalFormName', 'Societas Europaea'), ('marketCapitalization', '3500000000')]
[('euCountryName', 'Czech Republic'), ('legalFormName', 'akciová společnost'), ('marketCapitalization', '286200000000')]
[('euCountryName', 'Denmark'), ('legalFormName', 'aktieselskab'), ('marketCapitalization', '289600000000')]
[('euCountryName', 'Finland'), ('legalFormName', 'Bankaktiebolag'), ('marketCapitalization', '415000000000')]
[('euCountryName', 'Finland'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCapitalization', '26257000000')]
[('euCountryName', 'France'), ('legalFormName', 'S.A.'), ('marketCapitalization', '10203000000')]
[('euCountryName', 'France'), ('legalFormName', 'société anonyme'), ('marketCapitalization', '139000000000')]
[('euCountryName', 'Germany'), ('legalFormName', 'Aktiengesellschaft'), ('marketCapitalization', '139800000

17

In [61]:
# What is the avg in each country for a given legal form?
# I considered only "market capitalization"
queryString = """
SELECT ?euCountryName ?legalFormName (AVG(?marketCap) AS ?marketCapitalization)
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)

    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    ?company wdt:P1454 ?legalForm.
    ?legalForm <http://schema.org/name> ?legalFormName.

    ?company wdt:P2226 ?marketCap.

}
GROUP BY ?euCountryName ?legalFormName
ORDER BY ?euCountryName ?legalFormName
"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('legalFormName', 'Aktiengesellschaft'), ('marketCapitalization', '10960000000')]
[('euCountryName', 'Austria'), ('legalFormName', 'Societas Europaea'), ('marketCapitalization', '3500000000')]
[('euCountryName', 'Czech Republic'), ('legalFormName', 'akciová společnost'), ('marketCapitalization', '150772727272.727272727272727')]
[('euCountryName', 'Denmark'), ('legalFormName', 'aktieselskab'), ('marketCapitalization', '142452000000')]
[('euCountryName', 'Finland'), ('legalFormName', 'Bankaktiebolag'), ('marketCapitalization', '415000000000')]
[('euCountryName', 'Finland'), ('legalFormName', 'julkinen osakeyhtiö'), ('marketCapitalization', '26257000000')]
[('euCountryName', 'France'), ('legalFormName', 'S.A.'), ('marketCapitalization', '3068109916.166666666666667')]
[('euCountryName', 'France'), ('legalFormName', 'société anonyme'), ('marketCapitalization', '57864800000')]
[('euCountryName', 'Germany'), ('legalFormName', 'Aktiengesellschaft'), ('mar

17

### Point 5.2 End
DESCRIPTION: Using previous queries I tried to get more info about wdt:P3362 “operating income”, wdt:P2770 “source of income”, wdt:P2663 “common equity tier 1 capital ratio”, wdt:P2226 “market capitalization”, wdt:P4103 “asset under management”, wdt:P2403 “total asset”.
For the first part I decided to keep all above properties except for wdt:P2663. The result of the last query of subsection 5.2.1 shows lists all the companies that have at least one of these relations.
For the following parts I considered only market capitalization.


RESULT: The results are reported in the last three queries of this section. They show the min, max and average market capitalization for every European country for every legal form. Not all countries and not all legal forms are showed because for them no company has the market capitalization relation.

### Point 5.3 Start

In [64]:
# Return business that own businesses in other E.U. countries and how many they have
queryString = """
SELECT ?ownerCompany (COUNT(?ownedCompany) AS ?ownedCompanies)
WHERE { 

    ?ownerCompany wdt:P31 ?ownerCompanyType.
    ?ownerCompany <http://schema.org/name> ?ownerCompanyName.
    FILTER(?ownerCompanyType = wd:Q6881511 || ?ownerCompanyType = wd:Q4830453)
    
    ?ownedCompany wdt:P31 ?ownedCompanyType.
    ?ownedCompany wdt:P127 ?ownerCompany.
    FILTER(?ownedCompanyType = wd:Q6881511 || ?ownedCompanyType = wd:Q4830453)
    
    ?ownerEuCountry wdt:P463 wd:Q458.
    ?ownerCompany wdt:P17 ?ownerEuCountry.
    ?ownerEuCountry <http://schema.org/name> ?ownerEuCountryName.
    
    ?ownedEuCountry wdt:P463 wd:Q458.
    ?ownedCompany wdt:P17 ?ownedEuCountry.
    
    FILTER(?ownerEuCountry != ?ownedEuCountry)

}
GROUP BY ?ownerCompany
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('ownerCompany', 'http://www.wikidata.org/entity/Q61943664'), ('ownedCompanies', '4')]
[('ownerCompany', 'http://www.wikidata.org/entity/Q43929576'), ('ownedCompanies', '2')]
[('ownerCompany', 'http://www.wikidata.org/entity/Q63983613'), ('ownedCompanies', '4')]
[('ownerCompany', 'http://www.wikidata.org/entity/Q2558951'), ('ownedCompanies', '2')]
[('ownerCompany', 'http://www.wikidata.org/entity/Q60607807'), ('ownedCompanies', '4')]
[('ownerCompany', 'http://www.wikidata.org/entity/Q10854950'), ('ownedCompanies', '2')]
[('ownerCompany', 'http://www.wikidata.org/entity/Q20120422'), ('ownedCompanies', '12')]
[('ownerCompany', 'http://www.wikidata.org/entity/Q56684871'), ('ownedCompanies', '4')]
[('ownerCompany', 'http://www.wikidata.org/entity/Q98634662'), ('ownedCompanies', '1')]
[('ownerCompany', 'http://www.wikidata.org/entity/Q43907584'), ('ownedCompanies', '2')]


10

In [ ]:
# What can we say about industry sectors in various countries?
#For each country return the fields of work and how many companies works on it

queryString = """
SELECT DISTINCT ?euCountryName ?fieldOfWorkName  (COUNT(DISTINCT ?company) AS ?numberOfCompanies)
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.
    
    ?company wdt:P452 ?industry.
    ?industry wdt:P101|wdt:P425 ?fieldOfWork.
    
    ?fieldOfWork <http://schema.org/name> ?fieldOfWorkName .
    
    
}
ORDER BY ?euCountryName DESC(?numberOfCompanies)

"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'software development'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'optics'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'aluminium'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '1')]
[('euCountryName', 'Belgium'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '2')]
[('euCountryName', 'Bulgaria'), ('fieldOfWorkName', 'Travel agency activities'), ('numberOfCompanies', '1')]
[('euCountryName', 'Cyprus'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '1')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '13')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'dairying'), ('numberOfCompanies', '5')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'health care'), ('numberOfCompanies', '4')]
[('euCountryName

144

In [86]:
# Which business in each country owns more businesses in other E.U. countries?
queryString = """
SELECT ?euCountryName ?companyName ?ownedCompanies
WHERE {

    ?company wdt:P31 ?companyType.
    ?company <http://schema.org/name> ?companyName.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.

    {
        SELECT ?ownerCompany (COUNT(?ownedCompany) AS ?ownedCompanies)
        WHERE { 

            ?ownerCompany wdt:P31 ?ownerCompanyType.
            FILTER(?ownerCompanyType = wd:Q6881511 || ?ownerCompanyType = wd:Q4830453)

            ?ownedCompany wdt:P31 ?ownedCompanyType.
            ?ownedCompany wdt:P127 ?ownerCompany.
            FILTER(?ownedCompanyType = wd:Q6881511 || ?ownedCompanyType = wd:Q4830453)

            ?ownerEuCountry wdt:P31 wd:Q6256.
            ?ownerEuCountry wdt:P463 wd:Q458.
            ?ownerCompany wdt:P17 ?ownerEuCountry.

            ?ownedEuCountry wdt:P31 wd:Q6256.
            ?ownedEuCountry wdt:P463 wd:Q458.
            ?ownedCompany wdt:P17 ?ownedEuCountry.

            FILTER(?ownerEuCountry != ?ownedEuCountry)

        }
        GROUP BY ?ownerCompany
    }

    {
        SELECT ?euCountryNameIn (MAX(?ownedCompanies) AS ?ownsIn)
        WHERE {

            ?company wdt:P31 ?companyType.
            ?company <http://schema.org/name> ?companyName.
            FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)

            ?company wdt:P17 ?euCountry.
            ?euCountry <http://schema.org/name> ?euCountryNameIn.

            {
                SELECT ?ownerCompany (COUNT(?ownedCompany) AS ?ownedCompanies)
                WHERE { 

                    ?ownerCompany wdt:P31 ?ownerCompanyType.
                    FILTER(?ownerCompanyType = wd:Q6881511 || ?ownerCompanyType = wd:Q4830453)

                    ?ownedCompany wdt:P31 ?ownedCompanyType.
                    ?ownedCompany wdt:P127 ?ownerCompany.
                    FILTER(?ownedCompanyType = wd:Q6881511 || ?ownedCompanyType = wd:Q4830453)

                    ?ownerEuCountry wdt:P31 wd:Q6256.
                    ?ownerEuCountry wdt:P463 wd:Q458.
                    ?ownerCompany wdt:P17 ?ownerEuCountry.

                    ?ownedEuCountry wdt:P31 wd:Q6256.
                    ?ownedEuCountry wdt:P463 wd:Q458.
                    ?ownedCompany wdt:P17 ?ownedEuCountry.

                    FILTER(?ownerEuCountry != ?ownedEuCountry)

                }
                GROUP BY ?ownerCompany
            }

            FILTER(?company = ?ownerCompany)

        }
        GROUP BY ?euCountryNameIn
    }
    
    FILTER(?company = ?ownerCompany && ?ownedCompanies = ?ownsIn)
}
ORDER BY ?euCountryName

"""

print("Results")
run_query(queryString)

Results
The operation failed timed out


In [77]:
# Which business in each country owns more businesses in other E.U. countries?
queryString = """
SELECT DISTINCT ?euCountryNameOut ?companyNameOut (?ownedCompanies2 AS ?ownedCompanies)
WHERE {

    ?companyOut wdt:P31 ?companyTypeOut.
    ?companyOut <http://schema.org/name> ?companyNameOut.
    FILTER(?companyTypeOut = wd:Q6881511 || ?companyTypeOut = wd:Q4830453)

    ?euCountryOut wdt:P463 wd:Q458.
    ?companyOut wdt:P17 ?euCountryOut.
    ?euCountryOut <http://schema.org/name> ?euCountryNameOut.

    {
        SELECT DISTINCT ?ownerCompany2 (COUNT(?ownedCompany2) AS ?ownedCompanies2)
        WHERE { 

            ?ownerCompany2 wdt:P31 ?ownerCompanyType2.
            FILTER(?ownerCompanyType2 = wd:Q6881511 || ?ownerCompanyType2 = wd:Q4830453)

            ?ownedCompany2 wdt:P31 ?ownedCompanyType2.
            ?ownedCompany2 wdt:P127 ?ownerCompany2.
            FILTER(?ownedCompanyType2 = wd:Q6881511 || ?ownedCompanyType2 = wd:Q4830453)

            ?ownerEuCountry2 wdt:P463 wd:Q458.
            ?ownerCompany2 wdt:P17 ?ownerEuCountry2.

            ?ownedEuCountry2 wdt:P463 wd:Q458.
            ?ownedCompany2 wdt:P17 ?ownedEuCountry2.

            FILTER(?ownerEuCountry2 != ?ownedEuCountry2)

        }
        GROUP BY ?ownerCompany2
    }

    {
        SELECT DISTINCT ?euCountryNameIn (MAX(?ownedCompanies3) AS ?ownsIn)
        WHERE {

            ?company wdt:P31 ?companyType.
            ?company <http://schema.org/name> ?companyName.
            FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)

            ?company wdt:P17 ?euCountry.
            ?euCountry <http://schema.org/name> ?euCountryNameIn.

            {
                SELECT ?ownerCompany3 (COUNT(?ownedCompany3) AS ?ownedCompanies3)
                WHERE { 

                    ?ownerCompany3 wdt:P31 ?ownerCompanyType3.
                    FILTER(?ownerCompanyType3 = wd:Q6881511 || ?ownerCompanyType3 = wd:Q4830453)

                    ?ownedCompany3 wdt:P31 ?ownedCompanyType3.
                    ?ownedCompany3 wdt:P127 ?ownerCompany3.
                    FILTER(?ownedCompanyType3 = wd:Q6881511 || ?ownedCompanyType3 = wd:Q4830453)

                    ?ownerEuCountry3 wdt:P463 wd:Q458.
                    ?ownerCompany3 wdt:P17 ?ownerEuCountry3.

                    ?ownedEuCountry3 wdt:P463 wd:Q458.
                    ?ownedCompany3 wdt:P17 ?ownedEuCountry3.

                    FILTER(?ownerEuCountry3 != ?ownedEuCountry3)

                }
                GROUP BY ?ownerCompany3
            }

            FILTER(?company = ?ownerCompany3)

        }
        GROUP BY ?euCountryNameIn
    }
    
    FILTER(?companyOut = ?ownerCompany2 && ?ownedCompanies2 = ?ownsIn && ?euCountryNameIn = ?euCountryNameOut)
}
ORDER BY ?euCountryNameOut

"""

print("Results")
run_query(queryString)

Results
[('euCountryNameOut', 'Austria'), ('companyNameOut', 'Strabag'), ('ownedCompanies', '10')]
[('euCountryNameOut', 'Belgium'), ('companyNameOut', 'Euronext'), ('ownedCompanies', '20')]
[('euCountryNameOut', 'Croatia'), ('companyNameOut', 'Agrokor'), ('ownedCompanies', '4')]
[('euCountryNameOut', 'Croatia'), ('companyNameOut', 'Podravka'), ('ownedCompanies', '4')]
[('euCountryNameOut', 'Cyprus'), ('companyNameOut', 'Penta Investments Limited'), ('ownedCompanies', '14')]
[('euCountryNameOut', 'Czech Republic'), ('companyNameOut', 'ČEZ Group'), ('ownedCompanies', '12')]
[('euCountryNameOut', 'Czech Republic'), ('companyNameOut', 'J&T Finance Group'), ('ownedCompanies', '12')]
[('euCountryNameOut', 'Denmark'), ('companyNameOut', 'Euronext'), ('ownedCompanies', '20')]
[('euCountryNameOut', 'Estonia'), ('companyNameOut', 'Tallink'), ('ownedCompanies', '2')]
[('euCountryNameOut', 'Finland'), ('companyNameOut', 'Amer Sports'), ('ownedCompanies', '10')]
[('euCountryNameOut', 'France'), ('

30

### Point 5.3 End
DESCRIPTION: I started by returning the E.U. companies owned by E.U. companies in a different country and count the number of them. Then for every E.U. country I returned the number of owned companies by the company that owns the most.
The last query aims to also get the name of the companies.


RESULT: The result shows the list of the E.U. countries, the name of the company of that country that owns the most companies along with how many companies they own. 
Some countries have more entries because there is more than one company located in that country that owns the highest number of companies.
This happened for Croatia, Czech Republic, France and Hungary.

### Point 5.4 Start

In [91]:
# Check info about industry of companies in France
queryString = """
SELECT DISTINCT ?object ?objectName
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P17 wd:Q142.
    
    ?company wdt:P452 ?object.
    ?object <http://schema.org/name> ?objectName.
    
}
ORDER BY ?objectName
"""

print("Results")
run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q29584255'), ('objectName', '10  Manufacture of food products')]
[('object', 'http://www.wikidata.org/entity/Q29585034'), ('objectName', '11.01  Distilling, rectifying and blending of spirits')]
[('object', 'http://www.wikidata.org/entity/Q229367'), ('objectName', '3D printing')]
[('object', 'http://www.wikidata.org/entity/Q100319301'), ('objectName', '59  Motion picture, video and television programme production, sound recording and music publishing activities')]
[('object', 'http://www.wikidata.org/entity/Q100252087'), ('objectName', 'Accommodation and food service activities')]
[('object', 'http://www.wikidata.org/entity/Q29586050'), ('objectName', 'Activities of business and employers membership organisations')]
[('object', 'http://www.wikidata.org/entity/Q29584769'), ('objectName', 'Activities of holding companies')]
[('object', 'http://www.wikidata.org/entity/Q29585706'), ('objectName', 'Administration of financial markets')]
[(

514

In [94]:
# Get outgoing properties of industry of companies in France
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P17 wd:Q142.
    
    ?company wdt:P452 ?industry.
    ?industry ?property ?object.
    
    ?property <http://schema.org/name> ?propertyName .
    
    
}
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P8992'), ('propertyName', '1914-1918-Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7456'), ('propertyName', 'AAAS keyword ID')]
[('property', 'http://www.wikidata.org/prop/direct/P9084'), ('propertyName', 'ABC News topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2179'), ('propertyName', 'ACM Classification Code (2012)')]
[('property', 'http://www.wikidata.org/prop/direct/P6981'), ('propertyName', 'ACNP journal ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8061'), ('propertyName', 'AGROVOC ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8490'), ('propertyName', 'ANZSIC 2006 ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5922'), ('propertyName', 'ANZSRC 2008 FoR ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8529'), ('propertyName', 'ANZSRC 2020 FoR ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5984'), ('propertyName', 'APNI ID')]
[('proper

462

In [95]:
# Get info about "field of work" in industry
queryString = """
SELECT DISTINCT ?fieldOfWork ?fieldOfWorkName
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P17 wd:Q142.
    
    ?company wdt:P452 ?industry.
    ?industry wdt:P101 ?fieldOfWork.
    
    ?fieldOfWork <http://schema.org/name> ?fieldOfWorkName .
    
    
}
ORDER BY ?fieldOfWorkName
"""

print("Results")
run_query(queryString)

Results
[('fieldOfWork', 'http://www.wikidata.org/entity/Q64698773'), ('fieldOfWorkName', 'dairying')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q32635'), ('fieldOfWorkName', 'domain name')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q844861'), ('fieldOfWorkName', 'electric power distribution')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q31207'), ('fieldOfWorkName', 'health care')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q1540863'), ('fieldOfWorkName', 'information technology consulting')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q43183'), ('fieldOfWorkName', 'insurance')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q11030'), ('fieldOfWorkName', 'journalism')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q14620'), ('fieldOfWorkName', 'optics')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q64821189'), ('fieldOfWorkName', 'patient care')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q159719'), ('fieldOfWorkName', 'power station')]
[(

13

In [87]:
# Get info about "field of this occupation" in industry
queryString = """
SELECT DISTINCT ?fieldOfWork ?fieldOfWorkName
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?company wdt:P17 wd:Q142.
    
    ?company wdt:P452 ?industry.
    ?industry wdt:P425 ?fieldOfWork.
    
    ?fieldOfWork <http://schema.org/name> ?fieldOfWorkName .
    
    
}
ORDER BY ?fieldOfWorkName
"""

print("Results")
run_query(queryString)

Results
[('fieldOfWork', 'http://www.wikidata.org/entity/Q72854003'), ('fieldOfWorkName', 'hairdressing')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q57261067'), ('fieldOfWorkName', 'hairdressing industry')]
[('fieldOfWork', 'http://www.wikidata.org/entity/Q16879932'), ('fieldOfWorkName', 'musical instrument making')]


3

In [ ]:
# What can we say about industry sectors in various countries?
#For each country return the fields of work and how many companies works on it

queryString = """
SELECT DISTINCT ?euCountryName ?fieldOfWorkName  (COUNT(DISTINCT ?company) AS ?numberOfCompanies)
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.
    
    ?company wdt:P452 ?industry.
    ?industry wdt:P101|wdt:P425 ?fieldOfWork.
    
    ?fieldOfWork <http://schema.org/name> ?fieldOfWorkName .
    
    
}
ORDER BY ?euCountryName DESC(?numberOfCompanies)

"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'software development'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'optics'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'aluminium'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '1')]
[('euCountryName', 'Belgium'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '2')]
[('euCountryName', 'Bulgaria'), ('fieldOfWorkName', 'Travel agency activities'), ('numberOfCompanies', '1')]
[('euCountryName', 'Cyprus'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '1')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '13')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'dairying'), ('numberOfCompanies', '5')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'health care'), ('numberOfCompanies', '4')]
[('euCountryName

144

In [ ]:
# What can we say about industry sectors in various countries?
#For each country return the fields of work and how many companies works on it

queryString = """
SELECT DISTINCT ?euCountryName ?fieldOfWorkName  (COUNT(DISTINCT ?company) AS ?numberOfCompanies)
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.
    
    ?company wdt:P452 ?industry.
    ?industry wdt:P101|wdt:P425 ?fieldOfWork.
    
    ?fieldOfWork <http://schema.org/name> ?fieldOfWorkName .
    
    
}
ORDER BY ?euCountryName DESC(?numberOfCompanies)

"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'software development'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'optics'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'aluminium'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '1')]
[('euCountryName', 'Belgium'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '2')]
[('euCountryName', 'Bulgaria'), ('fieldOfWorkName', 'Travel agency activities'), ('numberOfCompanies', '1')]
[('euCountryName', 'Cyprus'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '1')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '13')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'dairying'), ('numberOfCompanies', '5')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'health care'), ('numberOfCompanies', '4')]
[('euCountryName

144

In [ ]:
# What can we say about industry sectors in various countries?
#For each country return the fields of work and how many companies works on it

queryString = """
SELECT DISTINCT ?euCountryName ?fieldOfWorkName  (COUNT(DISTINCT ?company) AS ?numberOfCompanies)
WHERE {

    ?company wdt:P31 ?companyType.
    FILTER(?companyType = wd:Q6881511 || ?companyType = wd:Q4830453)
    
    ?euCountry wdt:P463 wd:Q458.

    ?company wdt:P17 ?euCountry.
    ?euCountry <http://schema.org/name> ?euCountryName.
    
    ?company wdt:P452 ?industry.
    ?industry wdt:P101|wdt:P425 ?fieldOfWork.
    
    ?fieldOfWork <http://schema.org/name> ?fieldOfWorkName .
    
    
}
ORDER BY ?euCountryName DESC(?numberOfCompanies)

"""

print("Results")
run_query(queryString)

Results
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'software development'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'optics'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'aluminium'), ('numberOfCompanies', '1')]
[('euCountryName', 'Austria'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '1')]
[('euCountryName', 'Belgium'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '2')]
[('euCountryName', 'Bulgaria'), ('fieldOfWorkName', 'Travel agency activities'), ('numberOfCompanies', '1')]
[('euCountryName', 'Cyprus'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '1')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'insurance'), ('numberOfCompanies', '13')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'dairying'), ('numberOfCompanies', '5')]
[('euCountryName', 'Czech Republic'), ('fieldOfWorkName', 'health care'), ('numberOfCompanies', '4')]
[('euCountryName

144

### Point 5.4. End
DESCRIPTION: I used the previously discover relation “industry” wdt:P452 and tried it on French companies. Then I got all outgoing properties of industry instance and discover the “field of work” wdt:P101 and “field of this occupation” wdt:P425 relations. I discovered that the top 3 sectors in France are “insurance” wd:Q43183 with 10 companies, “dairying” wd:Q64698773 and  “software development” wd:Q638608 both with 4 companies.
Then I repeated the query considering all the E.U. countries


RESULT: The result was obtained by considering every field of work in every country and count how many companies works in that field. We can observe that the insurance field is present in almost all countries and is often in the first place. The top two fields of work for Italy are insurance and sound recording and music publishing activities.